In [1]:
from pyhdf.HDF import *
from pyhdf.VS import *
from pyhdf.SD import SD, SDC
import numpy as np

In [2]:
def get_normalized_bit(value, bit_index):
    return (value >> bit_index) & 1

In [3]:
#infile = '/glade/derecho/scratch/swei/Dataset/input/obs/calipso/CAL_LID_L2_05kmAPro-Standard-V4-20.2019-07-22T07-49-21ZD.hdf'
infile = '/glade/derecho/scratch/swei/Dataset/input/obs/calipso/CAL_LID_L1-Standard-V4-10.2019-07-22T07-49-21ZD.hdf'
hdf = HDF(infile)
vs = hdf.vstart()
metaid = vs.find('metadata')
vd = vs.attach(metaid)
vd.setfields('Lidar_Data_Altitudes')
alt = np.array(vd.read()[0][0])
vd.detach()
vs.end()

In [5]:
alt[:33]

array([39.79566956, 39.4962883 , 39.19690704, 38.89752197, 38.59814072,
       38.29875946, 37.9993782 , 37.69999695, 37.40061569, 37.10123444,
       36.80185318, 36.50247192, 36.20309067, 35.90370941, 35.60432434,
       35.30494308, 35.00556183, 34.70618057, 34.40679932, 34.10741806,
       33.8080368 , 33.50865555, 33.20927429, 32.90989304, 32.61051178,
       32.31112671, 32.01174545, 31.7123661 , 31.41298294, 31.11360168,
       30.81422043, 30.51483917, 30.21545792])

In [3]:
# Level 1 backscatter
sd = SD(infile, SDC.READ)

In [4]:
qc1 = sd.select('QC_Flag').get()
qc2 = sd.select('QC_Flag_2').get()

In [5]:
qc = qc1 + qc2

In [15]:
bit_list1_532 = [[1, 2], 4, 5, 7, [8, 9], [11, 12], 13, 15, 17, 19, 20, 22, 24]
bit_list2_532 = [[2, 3], [5, 6], [8, 9], [11, 12], [14, 15], [17, 18], [20, 21], [23, 24], 26]
bit_list_1_1064 = [3, 4, 6, 10, 14, 16, 18, 21, 23]
bit_list_2_1064 = [4, 7, 10, 13, 16, 19, 22, 25, 26, 27]

int

In [7]:
# Level 2 extinction
sd = SD(infile, SDC.READ)
data = sd.select('Extinction_Coefficient_532').get()
data2 = sd.select('Extinction_Coefficient_1064').get()
err = sd.select('Extinction_Coefficient_Uncertainty_532').get()
cad = sd.select('CAD_Score').get()

In [16]:
sd.select('CAD_Score').dimensions()

{'fakeDim75': 4192, 'fakeDim76': 399, 'fakeDim77': 2}

In [41]:
mask = data=-9999.
mask2 = (data2<0.)&(data2!=-9999.)

In [25]:
c = np.stack((data, data2), axis=1)

In [26]:
c.shape

(4192, 2, 399)

In [31]:
lats=hdf.select('Latitude').get()[:,1]
lats.shape

(4192,)

In [79]:
qcf = sd.select('Extinction_QC_Flag_1064').get()
tmpqcf = np.where(qcf[:, :, 0]==qcf[:, :, 1], qcf[:, :, 0], qcf.sum(axis=2))
tmpqcf.shape

(4192, 399)

In [80]:
qcf.shape

(4192, 399, 2)